In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [ ]:
def get_list_of_university_towns():
    uni_towns = open('university_towns.txt') 
    uni_towns_df = []
    state = None
    state_towns = []
    with uni_towns as file:
        for line in file:
            thisLine = line[:-1]
            if thisLine[-6:] == '[edit]':
                state = thisLine[:-6]
                continue
            if '(' in line:
                town = thisLine[:thisLine.index('(')-1]
                state_towns.append([state,town])
            else:
                town = thisLine
                state_towns.append([state,town])
            uni_towns_df.append(thisLine)
    df = pd.DataFrame(state_towns,columns = ['State','RegionName'])
    return df
get_list_of_university_towns()

In [ ]:
def get_recession_start():
    gdp = (pd.read_excel('gdplev.xls', skiprows=219, skipfooter=0).drop(labels=['Unnamed: 0','Unnamed: 1',
                                                                            'Unnamed: 2', 'Unnamed: 3', 12323.3,
                                                                                'Unnamed: 7'], axis=1))
    gdp = gdp.rename(index=str, columns={"1999q4": "Quarter", 9926.1: "GDP"})
    for i in range(2, len(gdp)):
        if (gdp.iloc[i-2][1] > gdp.iloc[i-1][1]) and (gdp.iloc[i-1][1] > gdp.iloc[i][1]):
            return gdp.iloc[i-2][0]
        
get_recession_start()

In [ ]:
def get_recession_end():
    gdp = (pd.read_excel('gdplev.xls', skiprows=219, skipfooter=0).drop(labels=['Unnamed: 0','Unnamed: 1',
                                                                            'Unnamed: 2', 'Unnamed: 3', 12323.3,
                                                                                'Unnamed: 7'], axis=1))
    gdp = gdp.rename(index=str, columns={"1999q4": "Quarter", 9926.1: "GDP"})
    start_index = gdp[gdp['Quarter'] == get_recession_start()].index.tolist()[0]
    gdp = gdp[start_index:]
    for i in range(2, len(gdp)):
        if (gdp.iloc[i-2][1] < gdp.iloc[i-1][1]) and (gdp.iloc[i-1][1] < gdp.iloc[i][1]):
            return gdp.iloc[i-2][0]
        
get_recession_end()

In [ ]:
def get_recession_bottom():
    gdp = (pd.read_excel('gdplev.xls', skiprows=219, skipfooter=0).drop(labels=['Unnamed: 0','Unnamed: 1',
                                                                            'Unnamed: 2', 'Unnamed: 3', 12323.3,
                                                                                'Unnamed: 7'], axis=1))
    gdp = gdp.rename(index=str, columns={"1999q4": "Quarter", 9926.1: "GDP"})
    start_index = gdp[gdp['Quarter'] == get_recession_start()].index.tolist()[0]
    end_index = gdp[gdp['Quarter'] == get_recession_end()].index.tolist()[0]
    
    min_gdp = gdp[start_index:end_index].min()[1]

    return gdp[gdp['GDP'] == min_gdp].iloc[0]['Quarter']

get_recession_bottom()

In [ ]:
def convert_housing_data_to_quarters():
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    home_data = pd.read_csv('City_Zhvi_AllHomes.csv').drop(['Metro','CountyName','RegionID','SizeRank'],axis=1)
    #map in the state names     
    home_data['State'] = home_data['State'].map(states)
    #set the index
    home_data.set_index(['State','RegionName'],inplace=True)

    drop_cols = list(home_data.columns)[0:45]
    home_data = home_data.drop(drop_cols,axis=1)

    home_data_q = (home_data.groupby(pd.PeriodIndex(home_data.columns, freq='Q'), axis=1)
                      .mean()
                      .rename(columns=lambda c: str(c).lower()))
    
    return home_data_q

In [ ]:
test_data = convert_housing_data_to_quarters().copy()
test_data = test_data.loc[:,'2008q3':'2009q2']
test_data['Delta'] = (test_data['2009q2']-test_data['2008q3'])/test_data['2008q3']
test_data = test_data.reset_index()

uni_town = pd.DataFrame(get_list_of_university_towns()['RegionName'])
uni_town['uni_town'] = 1
uni_town = uni_town.drop_duplicates()

merged_data = pd.merge(test_data, uni_town, on='RegionName', how='left')
#merged_data.set_index(['State','RegionName'],inplace=True)
merged_data = merged_data.fillna(0)

not_uni = merged_data[merged_data['uni_town']==0].loc[:,'Delta'].dropna()
is_uni  = merged_data[merged_data['uni_town']==1].loc[:,'Delta'].dropna()

if not_uni.mean() > is_uni.mean():
    better = 'non-university town'
else:
    better = 'university town'

if p_val < .05:
    significant = True
else:
    significant = False

p_val = list(ttest_ind(not_uni, is_uni))[1]

result = (significant,p_val,better)

print(result)